# Importando bibliotecas


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Importando base de dados


In [ ]:
train_set = pd.read_csv(r'C:\Users\jpcar\Documents\DL_Projects\adult-pmr3508\train_data.csv', na_values="?")
test_set = pd.read_csv(r'C:\Users\jpcar\Documents\DL_Projects\adult-pmr3508\test_data.csv', na_values="?")

# Pré-processamento dos dados


In [ ]:
#amostra das 5 primeiras linhas do grupo de teste
train_set.head()

In [ ]:
#amostra das 5 primeiras linhas do grupo de teste
test_set.head()

In [ ]:
#verificando valores nulos na base de treino
train_set.isnull().sum()

In [ ]:
#deletando valores nulos encontrados
train_set = train_set.dropna()

In [ ]:
#re-verificando se restou valores nulos
train_set.isnull().sum()

In [ ]:
#Retirando coluna de Id e fnlwgt do treino e teste
train_set = train_set.drop(columns=["Id", "fnlwgt"])
test_set = test_set.drop(columns=["Id", "fnlwgt"])

In [ ]:
#Separando as colunas para relacionar atributos de X de treino com Y de treino
X_train = train_set[train_set.columns.to_list()]
X_train = X_train.drop(columns=["income"])
y_train = train_set[["income"]]
X_test = test_set[test_set.columns.to_list()]

In [ ]:
#Realizando a normalização dos dados para não haver desbalanceamento entre as classes
def encode_labels(df: pd.DataFrame) -> pd.DataFrame:
    return df.apply(LabelEncoder().fit_transform)

def standardization(df: pd.DataFrame) -> pd.DataFrame:
    return StandardScaler().fit_transform(df)

def preprocess(original: pd.DataFrame):
    df = original.copy()
    # Aplica o label enconder a todos os valores do dataframe 
    df = encode_labels(df)
    # Normaliza os valores obtidos anteriormente
    np_array = standardization(df)
    result = pd.DataFrame(np_array, columns=original.columns)
    return result

X_train = preprocess(X_train)
X_test = preprocess(X_test)
y_train = encode_labels(y_train)
display(X_train.head())
display(y_train.head())

# Modelo usando KNN


In [ ]:
#Selecionando a vizinhança a ser verificados
knn = KNeighborsClassifier()
distributions = dict(n_neighbors = [9, 15, 18, 21, 24, 28, 33, 37, 42, 49, 51, 59])
classifier = GridSearchCV(knn, distributions)

In [ ]:
#Verificação dos melhores parametros
knn_search = classifier.fit(X_train, y_train)
knn_search.best_params_

In [ ]:
# acurácia do modelo
knn_search.best_score_

# Realizando Predição

In [ ]:
#aplicando no grupo de testes
results = knn_search.best_estimator_.predict(X_test)

In [ ]:
X_test.head()

In [ ]:
def to_income_string(item):
  return '<=50K' if item == 0 else '>50K'

predictions = [to_income_string(result) for result in results]
predictions = np.array(predictions, dtype='object')

In [ ]:
predictions

# Resultados 

In [ ]:
#atribuindo rótulos de id e de classificação "income" para o arquivo final
submissions = pd.DataFrame()
submissions[0] = test_set.index
submissions[1] = predictions
submissions.columns = ['Id', 'income']

In [ ]:
#visualizando tabela final de predição
submissions.head(n=10)

In [ ]:
#gerando arquivo para submissão
submissions.to_csv('submission.csv', index=False)